In [2]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt    
import seaborn as sns    
import re   
import os   
import warnings
warnings.filterwarnings("ignore")

In [12]:
with open("WhatsApp Chat with CodeAcuity.txt","r",encoding="utf-8") as fi:
    text = fi.read()

In [16]:
text

'3/22/24, 6:03\u202fPM - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n1/4/24, 4:10\u202fPM - Sai Kiran Code Acuity created group "CodeAcuity"\n3/22/24, 6:03\u202fPM - Sagar Code Acuity added you\n3/22/24, 6:57\u202fPM - Sagar Code Acuity: Ssuuupppp People????? I hope you had a good day, let’s make it the best by joining the party at Brad Ford Guest house tonight, and the party starts at 8:30 PM… 🍻🍻🍻🍻🍻🍻🍻🍻🍻 <This message was edited>\n3/22/24, 7:05\u202fPM - Tinku Code Acuity: Jojo Bday week crazy undhi kada, 2 confirmations ochinayi\n3/23/24, 2:42\u202fPM - Sai Kiran Code Acuity: <Media omitted>\n3/28/24, 6:22\u202fPM - Sagar Code Acuity: Hello People, Due to the observance of Good Friday, you guys can work from home tomorrow and there is no need to come to the office, Thank you.\n3/29/24, 12:13\u202fAM - Sagar Code Acuity: Please don’t assume or consider it as long weekend holiday, wake up at

In [46]:
def retrieve_date_and_time(dat):
    p = dat.split(", ")
    p = list(map(lambda x: x.strip(),p))
    date = p[0]
    time_1 = p[1].split(" ")
    time_part = time_1[0]
    meridian = time_1[1]

    time_divisions = [int(i) for i in time_part.split(":")]

    if meridian == "AM" and time_divisions[0] == 12:
        time_divisions[0] = 0
    elif meridian == "PM" and time_divisions[0] >= 1 and time_divisions[0] < 12:
        time_divisions[0] += 12

    
    hour = time_divisions[0]
    minute = time_divisions[1]

    

    final_time = hour + ":" + minute

    return final_time


retrieve_date_and_time("3/22/24, 1:03 AM")

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [30]:
pattern = "\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[A-Z]M\s-\s"

dates_1 = re.findall(pattern=pattern,string=text)
dates_2 = list(map(lambda x: x.replace("\u202f", " "),dates_1))
dates_3 = list(map(lambda x: x.split("-")[0].strip(),dates_2))

dates_3

['3/22/24, 6:03 PM',
 '1/4/24, 4:10 PM',
 '3/22/24, 6:03 PM',
 '3/22/24, 6:57 PM',
 '3/22/24, 7:05 PM',
 '3/23/24, 2:42 PM',
 '3/28/24, 6:22 PM',
 '3/29/24, 12:13 AM',
 '4/1/24, 2:43 PM',
 '4/1/24, 2:43 PM',
 '4/4/24, 7:16 PM',
 '4/8/24, 9:23 PM',
 '4/8/24, 10:51 PM',
 '4/9/24, 1:05 PM',
 '4/9/24, 1:06 PM',
 '4/11/24, 10:08 AM',
 '4/11/24, 3:34 PM',
 '4/16/24, 10:27 AM',
 '4/17/24, 5:27 PM',
 '4/23/24, 12:08 PM',
 '4/23/24, 12:09 PM',
 '4/23/24, 12:09 PM',
 '4/23/24, 12:10 PM',
 '4/23/24, 12:10 PM',
 '4/23/24, 12:10 PM',
 '4/23/24, 12:11 PM',
 '4/23/24, 12:13 PM',
 '4/23/24, 12:15 PM',
 '4/23/24, 12:17 PM',
 '4/23/24, 12:23 PM',
 '4/23/24, 12:24 PM',
 '4/23/24, 12:26 PM',
 '4/24/24, 10:35 AM',
 '4/24/24, 4:11 PM',
 '4/24/24, 4:11 PM',
 '5/2/24, 1:23 PM',
 '5/2/24, 1:23 PM',
 '5/2/24, 1:23 PM',
 '5/2/24, 1:25 PM',
 '5/3/24, 5:56 PM',
 '5/3/24, 6:00 PM',
 '5/7/24, 7:18 PM',
 '5/7/24, 9:05 PM',
 '5/7/24, 9:06 PM',
 '5/14/24, 3:37 PM',
 '5/14/24, 3:38 PM',
 '5/27/24, 9:12 PM',
 '6/10/24, 1